In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.arff import loadarff
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

In [2]:
raw_data, meta = loadarff('datasetsCBR/mushroom/mushroom.fold.000000.train.arff')
train_df = pd.DataFrame(raw_data, columns=meta.names())
train_df.head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,b'x',b's',b'w',b't',b'l',b'f',b'c',b'b',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'n',b'm',b'e'
1,b'k',b'y',b'e',b'f',b's',b'f',b'c',b'n',b'b',b't',...,b'p',b'w',b'p',b'w',b'o',b'e',b'w',b'v',b'd',b'p'
2,b'x',b'y',b'y',b't',b'a',b'f',b'c',b'b',b'n',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b's',b'm',b'e'
3,b'x',b'y',b'w',b't',b'p',b'f',b'c',b'n',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'v',b'u',b'p'
4,b'x',b'y',b'n',b'f',b'f',b'f',b'c',b'n',b'b',b't',...,b'w',b'p',b'p',b'w',b'o',b'e',b'w',b'v',b'l',b'p'


In [3]:
for col in train_df.columns:
    label_encoder = LabelEncoder()
    train_df[col] = label_encoder.fit_transform(train_df[col])
train_df.head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,5,2,8,1,3,1,0,0,4,0,...,7,7,0,2,1,4,2,2,3,0
1,3,3,2,0,7,1,0,1,0,1,...,6,7,0,2,1,0,7,4,0,1
2,5,3,9,1,0,1,0,0,5,0,...,7,7,0,2,1,4,2,3,3,0
3,5,3,8,1,6,1,0,1,4,0,...,7,7,0,2,1,4,2,4,5,1
4,5,3,4,0,2,1,0,1,0,1,...,7,6,0,2,1,0,7,4,2,1
